# Хорин Роман (ИАД-2)

## Классификация имен

In [1]:
import collections
from string import ascii_uppercase, ascii_lowercase
import nltk
from random import shuffle
from nltk.util import ngrams
from nltk import classify, NaiveBayesClassifier
import numpy as np

### #1. Предварительная обработка данных. [1 балл]

<p>Проведем небольшую предобработку данных.</p>
Создадим тренировочную выборку имен. После считываения всех имен определим, какие относятся к обоим полам, и удалим их. Сформируем тестовую выборку из 20% случайных имен, начинающихся с каждой буквы алфавита и удалим такие имена из тренировочной выборки.

In [2]:
# загружаем данные и формируем датасет из мужских и женских имен без дупликаций
with open("male.txt") as f:
    m_names = f.readlines()

with open("female.txt") as f:
    f_names = f.readlines()
    
all_names = m_names + f_names

# определяем, какие имена являются одновременно мужскими и женскими
mf_names = []
for f_name in f_names:
    if f_name in m_names:
        mf_names.append(f_name)
        
for m_name in m_names:
    if m_name in f_names:
        mf_names.append(m_name)

Распарсенные имена выглядят следующим образом.

In [3]:
m_names[:5]

['Aamir\n', 'Aaron\n', 'Abbey\n', 'Abbie\n', 'Abbot\n']

Пометим имена полом, к которому они относятся, и разделим выборку на тренировочную и тестовую.

In [8]:
# оставляем только уникальные мужские и женские имена
labeled_m_names = [(m_name.strip(), 'male') for m_name in m_names if not m_name in mf_names]
labeled_f_names = [(f_name.strip(), 'female') for f_name in f_names if not f_name in mf_names]
    
# формируем тренировочную выборку
train_labeled_names = labeled_m_names + labeled_f_names 
train_labeled_names.sort()
shuffle(train_labeled_names)

# формируем тестовую выборку имен
test_labeled_names = []
# for letter in ascii_lowercase:
for letter in ascii_uppercase:
    names_by_letter = [value for value in train_labeled_names if value[0][0].startswith(letter)]
    names_to_append = names_by_letter[:round(len(names_by_letter)*0.2)]
    test_labeled_names += names_to_append

# удаляем имена для теста из тренировочной выборки
for elem in test_labeled_names:
    train_labeled_names.remove(elem)

print('Preprocessing finished!')

Preprocessing finished!


Теперь наши данные с метками выглядят так.

In [9]:
train_labeled_names[:5]

[('Xever', 'male'),
 ('Quintina', 'female'),
 ('Elayne', 'female'),
 ('Teressa', 'female'),
 ('Evangelia', 'female')]

### #2. Используйте метод наивного Байеса для классификации имен. Сравните результаты, получаемые при разных n = 2, 3, 4 по F-мере и аккуратности. [4 балла]

На основе 2,3 и 4-символьных грамм обучим модели и определим, которая из них является лучшей по аккуратности и F-мере. 

In [4]:
"""Метод генерирует символьные н-грамы для слова"""
def generate_ngrams(word, n):
    return dict([(ngram, True) for ngram in list(ngrams(word, n))])

"""Метод формирует из слов признаки на основе символьных н-грам"""
def generate_featureset(names, n):
    return [(generate_ngrams(name, n), gender) for (name, gender) in names]

Данными, на которых обучается классификатор, являются слова, представленные в виде 2,3 или 4-символьных грам.

In [245]:
print('Models testing started...\n')

models_dic = {}

for number in [2, 3, 4]:
    print('Number of n-grams:',number)
    
    # формируем сеты признаков на основе символьных н-грам
    train_featureset = generate_featureset(train_labeled_names, number)
    test_featureset = generate_featureset(test_labeled_names, number)
    
    # добавляем в словарь
    models_dic['%s-gram_test_featurset'%number] = test_featureset
    
    # обучаем классификатор и также добавляем его в словарь
    clf = NaiveBayesClassifier.train(train_featureset)
    models_dic['%s-gram_model'%number] = clf
    
    # Формируем сеты для рассчета показателей
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
 
    for i, (feats, label) in enumerate(test_featureset):
        refsets[label].add(i)
        observed = clf.classify(feats)
        testsets[observed].add(i)  
    
    # рассчитываем F-меры и accuracy
    accuracy = nltk.classify.accuracy(clf, test_featureset)
    female_f_measure = nltk.f_measure(refsets['female'], testsets['female'])
    male_f_measure = nltk.f_measure(refsets['male'], testsets['male'])

    print('Female f_measure for featureset based on %s-grams is ->%f<-'%(number, female_f_measure))
    print('Male f_measure for featureset based on %s-grams is ->%f<-'%(number, male_f_measure))
    print('Accuracy for featureset based on %s-grams is ->%f<-\n'%(number, accuracy))

print('Models testing finished!')

Models testing started...

Number of n-grams: 2
Female f_measure for featureset based on 2-grams is ->0.843299<-
Male f_measure for featureset based on 2-grams is ->0.676596<-
Accuracy for featureset based on 2-grams is ->0.788889<-

Number of n-grams: 3
Female f_measure for featureset based on 3-grams is ->0.871236<-
Male f_measure for featureset based on 3-grams is ->0.740042<-
Accuracy for featureset based on 3-grams is ->0.827778<-

Number of n-grams: 4
Female f_measure for featureset based on 4-grams is ->0.853119<-
Male f_measure for featureset based on 4-grams is ->0.672646<-
Accuracy for featureset based on 4-grams is ->0.797222<-

Models testing finished!


Как видно из отчета, модель, обученная на основе 3-символьных грамм слов является лучшей.

In [84]:
# лучшая модель
best_bayess = models_dic['3-gram_model']

# тестовый featureset для этой модели
test_featureset = models_dic['3-gram_test_featurset']

Посмотрим, в каких случаях наша модель ошиблась.

In [86]:
print('Errors for best classification model:')
bayes_errors = []
for (name, gender) in test_labeled_names:
    prediction = best_bayess.classify(generate_ngrams(name, 3))
    if prediction != gender:
        bayes_errors.append((gender, prediction, name))
for (gender, prediction, name) in sorted(bayes_errors):
     print('\tcorrect={:<8} prediction={:<8s} name={:<30}'.format(gender, prediction, name))

Errors for best classification model:
	correct=female   prediction=male     name=Alayne                        
	correct=female   prediction=male     name=Alfreda                       
	correct=female   prediction=male     name=Alpa                          
	correct=female   prediction=male     name=Anatola                       
	correct=female   prediction=male     name=Andromache                    
	correct=female   prediction=male     name=Antonella                     
	correct=female   prediction=male     name=Aphrodite                     
	correct=female   prediction=male     name=Appolonia                     
	correct=female   prediction=male     name=Atlante                       
	correct=female   prediction=male     name=Augusta                       
	correct=female   prediction=male     name=Barbabra                      
	correct=female   prediction=male     name=Benedicta                     
	correct=female   prediction=male     name=Benoite                       


Несмотря на ошибки, наша модель в ~80% случаев правильно классифицирует пол, что является довольно-таки неплохим результатом.

### #3. Используйте сеть с двумя слоями LSTM для определения пола. Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по F-мере и аккуратности. [4 балла]

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import model_from_json

In [11]:
"""Метод преобразует имя в бинарный вектор размерностью (количество букв в алфавите X максимальная длина имени)
   где на месте буквы имени в матрице ставится единица."""
def convert_name_to_vector(name, charslen, maxlen, char_indices):
    x = np.zeros((charslen, maxlen), dtype=int)
    for char in name:
        if char in ascii_lowercase:
            x[char_indices[char]][0] = 1
    return x

"""Метод формирует данные для нейросети"""
def generate_data(labeled_names, charslen, maxlen, char_indices):
    vectored_names = []
    for (name, gender) in labeled_names:
        x = convert_name_to_vector(name.lower(), charslen, maxlen, char_indices)
        vectored_names.append(x)
    return np.array(vectored_names)

"""Метод формирует метки данных для нейросети"""
def generate_labels(labeled_names):
    y = []
    for (name, gender) in labeled_names:
        label_to_vec = np.zeros(2, )
        if gender == 'male':
            label_to_vec[0] = 1
        else:
            label_to_vec[1] = 1
        y.append(label_to_vec)

    return np.array(y)

Осуществим преобразование тренировочных и тестовых данных в соответствии с заданием при помощи методов, написанных выше.

In [12]:
char_indices = dict((c, i) for i, c in enumerate(ascii_lowercase))
charslen = len(ascii_lowercase) # длина алфавита
maxlen = len(max(m_names + f_names, key=len)) # максимальная длина слова

X_train = generate_data(train_labeled_names, charslen, maxlen, char_indices)
X_test = generate_data(test_labeled_names, charslen, maxlen, char_indices)

y_train = generate_labels(train_labeled_names)
y_test = generate_labels(test_labeled_names)

In [17]:
charslen, maxlen

(26, 16)

Выборки представляют из себя совокупность слов, каждое из которых является матрицей из нулей и единиц.
<p>Вот так выглядит, например, первое имя в тренировочной выборке:</p>

In [13]:
train_labeled_names[0][0], convert_name_to_vector(train_labeled_names[0][0].lower(), charslen, maxlen, char_indices)

('Xever', array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     

Сравним результаты, которые получаются при разных значениях дропаута и количества нейронов в узлах нейронной сети по F-мере и аккуратности.
Каждая из сетей будет имеет 2 слоя LSTM, а функцией активацией для выходного слоя будет являться softmax.

In [18]:
X_train.shape, y_train.shape

((5774, 26, 16), (5774, 2))

In [22]:
X_train[0].shape, y_train[0].shape

((26, 16), (2,))

In [235]:
%%time
neurons_and_dropout_values = [[100, 0.1], [512, 0.5], [260, 0.9]]
networks_dic = {}
for values in neurons_and_dropout_values:
    neurons = values[0]
    dropout_ratio = values[1]
    
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(charslen, maxlen)))
    model.add(Dropout(dropout_ratio))
    model.add(LSTM(neurons))
    model.add(Dropout(dropout_ratio))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'fmeasure'])
    
    # json_string = model.to_json()
    
    # with open("model_softmax_X&y_by_task.json", "w") as text_file:
    #     text_file.write(json_string)
    print('Training network with %d neurons and dropout=%.2f'%(neurons, dropout_ratio))
    model.fit(X_train, y_train, batch_size=50, nb_epoch=10, verbose=0)
    
    networks_dic['%d-neurons'%neurons] = model
#     print('Model trained and weights saved')
    # model.save_weights('model_softmax_X&y_by_task.h5')
    
    scores = model.evaluate(X_test, y_test, batch_size=50, verbose=0)
    print('Results for neurons=%d and droupout=%.2f:\n\t%s = %.2f%%\n\t%s = %.2f\n'%(neurons, dropout_ratio, model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]))

Training network with 100 neurons and dropout=0.10
Results for neurons=100 and droupout=0.10:
	acc = 65.49%
	fmeasure = 0.65

Training network with 512 neurons and dropout=0.50
Results for neurons=512 and droupout=0.50:
	acc = 67.64%
	fmeasure = 0.68

Training network with 260 neurons and dropout=0.90
Results for neurons=260 and droupout=0.90:
	acc = 65.83%
	fmeasure = 0.66

Wall time: 45min 18s


Можно заметить, что при 512 нейронах в слоях сети и показателем дропаута = 0.5 получается лучший результат из остальных. Наша сеть в ~68% случаев правильно классифицирует пол.

In [236]:
best_net = networks_dic['512-neurons']

In [23]:
charslen, maxlen

(26, 16)

Посмотрим, на каких именах наша сеть ошибочно классифицировала пол.

In [237]:
net_errors = []
for (name, gender) in test_labeled_names:
    name_vec = convert_name_to_vector(name.lower(), charslen, maxlen, char_indices)
    name_vec.shape = (1, charslen, maxlen)
    v = best_net.predict(name_vec).ravel()
    if v[0] > v[1]:
        prediction = "male"
    else:
        prediction = "female"

    if prediction != gender:
        net_errors.append((gender, prediction, name))
        
#     print('Name: %s   Gender: %s   Prediction: %s'%(name, gender, prediction))
        
print('Ошибки сети:')
for (gender, prediction, name) in sorted(net_errors):
     print('\tcorrect={:<8} prediction={:<8} name={:<30}'.format(gender, prediction, name))

Ошибки сети:
	correct=female   prediction=male     name=Cassandry                     
	correct=female   prediction=male     name=Constance                     
	correct=female   prediction=male     name=Consuela                      
	correct=female   prediction=male     name=Cordey                        
	correct=female   prediction=male     name=Cordie                        
	correct=female   prediction=male     name=Cristine                      
	correct=female   prediction=male     name=Delores                       
	correct=female   prediction=male     name=Devora                        
	correct=female   prediction=male     name=Dolores                       
	correct=female   prediction=male     name=Dorice                        
	correct=female   prediction=male     name=Elvera                        
	correct=female   prediction=male     name=Endora                        
	correct=female   prediction=male     name=Eudora                        
	correct=female   predict

### #4. Сравните результаты классификации разными методами. Какой метод лучше и почему? [1 балл]

<p>Из полученных выше результатов можно сделать следующий вывод:</p>
Наивный байесовский классификатор значительно превосходит по качеству построенную мной нейросеть (80% против 68% по accuracy соответственно). Это может быть связано с "наивным" подходом модели о независимости признаков.

<p>Неудача нейросети может быть объяснена несовершенностью её топологии, а также малым количеством к тому же несбалансированных данных, из-за чего она не может обнаружить необходимые зависимости.</p>

<p>Справиться с неудачей могло бы помочь увеличение количества данных и их балансировка, а также настройка гиперпараметров сети на кросс-валидации.</p>